In [1]:
import sys, os
sys.path.append(os.path.abspath('../..'))

In [2]:
from tqdm.notebook import tqdm
import math
import gym
import torch
import torch.optim as optim 
from torch.utils.tensorboard import SummaryWriter
from collections import deque

from networks.dqn_atari import ENS_DQN
from utils.memory import RankedReplayMemory, LabeledReplayMemory
from utils.optimization import AMN_perc_optimization, AMN_optimization_epochs
from environments.atari_wrappers import make_atari, wrap_deepmind
from utils.atari_utils import fp, ActionSelector, evaluate
from utils.acquisition_functions import mc_ramdom_ens

import imageio
from utils.visualization import visualize_AMN

In [3]:
env_name = 'Breakout'
env_raw = make_atari('{}NoFrameskip-v4'.format(env_name))
env = wrap_deepmind(env_raw, frame_stack=False, episode_life=False, clip_rewards=True)
c,h,w = c,h,w = fp(env.reset()).shape
n_actions = env.action_space.n

In [4]:
BATCH_SIZE = 64
LR = 0.0000625
GAMMA = 0.99
EPS = 0.05
NUM_STEPS = 10000000
NOT_LABELLED_CAPACITY = 1000
LABELLED_CAPACITY = 1000
INITIAL_STEPS=NOT_LABELLED_CAPACITY
EPOCHS=1
PERCENTAGE = 1

NAME = 'Ensemble_hypothesis'

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # if gpu is to be used
# AMN_net = MC_DQN(n_actions).to(device)
AMN_net = ENS_DQN(n_actions).to(device)
expert_net = torch.load("models/dqn_expert_breakout_model").to(device)
AMN_net.apply(AMN_net.init_weights)
expert_net.eval()
# optimizer = optim.Adam(AMN_net.parameters(), lr=LR, eps=1.5e-4)
optimizer = optim.Adam(AMN_net.parameters(), lr=LR, eps=1.5e-4)

In [6]:
# memory = RankedReplayMemory(M_SIZE, [5,h,w], n_actions, mc_BALD_ens, AMN_net, replacement=False, device=device)
memory = LabeledReplayMemory(NOT_LABELLED_CAPACITY, LABELLED_CAPACITY, [5,h,w], n_actions, mc_ramdom_ens, AMN_net, device=device)
action_selector = ActionSelector(EPS, EPS, AMN_net, 1, n_actions, device)

In [7]:
steps_done = 0
writer = SummaryWriter(f'runs/{NAME}')

In [8]:
q = deque(maxlen=5)
done=True
eps = 0
episode_len = 0
num_labels = 0

In [11]:
progressive = tqdm(range(NUM_STEPS), total=NUM_STEPS, ncols=400, leave=False, unit='b')
for step in progressive:
  if done:
    env.reset()
    sum_reward = 0
    episode_len = 0
    img, _, _, _ = env.step(1) # BREAKOUT specific !!!
    for i in range(10): # no-op
      n_frame, _, _, _ = env.step(0)
      n_frame = fp(n_frame)
      q.append(n_frame)
        
  # Select and perform an action
  state = torch.cat(list(q))[1:].unsqueeze(0)
  action, eps = action_selector.select_action(state)
  n_frame, reward, done, info = env.step(action)
  n_frame = fp(n_frame)

  # 5 frame as memory
  q.append(n_frame)
  memory.push(torch.cat(list(q)).unsqueeze(0), action, reward, done) # here the n_frame means next frame from the previous time step
  episode_len += 1

  # Perform one step of the optimization (on the target network)
  if step % NOT_LABELLED_CAPACITY == 0 and step > INITIAL_STEPS:
    num_labels += memory.label_sample(percentage=PERCENTAGE)
    loss = AMN_optimization_epochs(AMN_net, expert_net, optimizer, memory, EPOCHS,
                                   batch_size=BATCH_SIZE, device=device)
    if loss is not None:
      writer.add_scalar('loss_vs_#labels', loss, num_labels)
    evaluated_reward = evaluate(step, AMN_net, device, env_raw, n_actions, eps=0.05, num_episode=15)
    writer.add_scalar('reward_vs_#labels', evaluated_reward, num_labels)
  if step % 500000:
    torch.save(AMN_net, f'models/hypothesis_{step}')

RuntimeError: write(): fd 66 failed with No space left on device

In [ ]:
images = visualize_AMN(AMN_net,memory,expert_network=expert_net, device=device)
imageio.mimsave(f'gifs/{NAME}.gif', images)